# Exetera Import Example

This example demonstrates creating a simple database with a RandomDataset schema then converting it to a Exetera databse with its JSON schema. 

First thing to do is install and import `exetera` and `RandomDataset`:

In [1]:
%pip install RandomDataset exetera

     |████████████████████████████████| 91 kB 4.3 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import sys

import numpy as np
import pandas as pd

import randomdataset
import exetera, exetera.core, exetera.processing

The YAML schema is written out which will be used to generate the random data. This schema states the structure of the tables and how many lines to generate. What is generated is defined by which type creates a field with the arguments passed to that type's constructor. What type of file to generate is defined by the generator type, in this case `CSVGenerator`:

In [3]:
%%writefile randomschema.yml

- typename: randomdataset.generators.CSVGenerator
  num_lines: 10
  dataset:
    name: participants
    typename: randomdataset.Dataset
    fields:
    - name: id
      typename: randomdataset.UIDFieldGen
    - name: FirstName
      typename: randomdataset.StrFieldGen
      lmin: 6
      lmax: 14
    - name: LastName
      typename: randomdataset.StrFieldGen
      lmin: 6
      lmax: 14
    - name: Age
      typename: randomdataset.IntFieldGen
      vmin: 18
      vmax: 90
    - name: is_employed
      typename: randomdataset.BoolFieldGen
      as_string: False
        
- typename: randomdataset.generators.CSVGenerator
  num_lines: 10
  dataset:
    name: tests
    typename: randomdataset.Dataset
    fields:
    - name: id
      typename: randomdataset.UIDFieldGen
    - name: patient_id
      typename: randomdataset.IntFieldGen
      vmin: 0
      vmax: 10
    - name: test_type
      typename: randomdataset.SetFieldGen
      field_type: str
      values: ["Type1", "Type2", "Unknown"]
    - name: location
      typename: randomdataset.StrFieldGen
    - name: result
      typename: randomdataset.IntFieldGen
      vmin: 0
      vmax: 3
    - name: value
      typename: randomdataset.FloatFieldGen
      vmin: 0
      vmax: 9

Writing randomschema.yml


Instead of invoking the `generate_dataset` command line utility the command can be called directly through the imported library:

In [4]:
# !generate_dataset randomschema.yaml .
randomdataset.generate_dataset.callback("randomschema.yml",".")

Schema: 'randomschema.yml'
Output: '.'


Next the Exetera schema is written out, which looks like the YAML schema except with some extra specifiers for primary and foreign keys:

In [5]:
%%writefile exeteraschema.json

{
  "exetera": {
    "version": "1.0.0"
  },
  "schema": {
    "participants": {
      "primary_keys": [
        "id"
      ],
      "fields": {
        "id": {
          "field_type": "fixed_string",
          "length": 32
        },
        "FirstName": {
          "field_type": "fixed_string",
          "length": 32
        },
        "LastName": {
          "field_type": "fixed_string",
          "length": 32
        },
        "Age": {
          "field_type": "numeric",
          "value_type": "int32"
        },
        "is_employed": {
          "field_type": "categorical",
          "categorical": {
            "value_type": "int8",
            "strings_to_values": {
              "False": 1,
              "True": 2
            }
          }
        }
      }
    },
    "tests": {
      "primary_keys": [
        "id"
      ],
      "foreign_keys": {
        "participants_id": {
          "space": "participants",
          "key": "id"
        }
      },
      "fields": {
        "id": {
          "field_type": "fixed_string",
          "length": 32
        },
        "patient_id": {
          "field_type": "fixed_string",
          "length": 32
        },
        "test_type": {
          "field_type": "categorical",
          "categorical": {
            "value_type": "int8",
            "strings_to_values": {
              "Unknown": 0,
              "Type1": 1,
              "Type2": 2
            }
          }
        },
        "location": {
          "field_type": "string"
        },
        "result": {
          "field_type": "numeric",
          "value_type": "int32"
        },
        "value": {
          "field_type": "numeric",
          "value_type": "float32"
        } 
      }
    } 
  }
}


Writing exeteraschema.json


The conversion command is run to produce the HDF5 dataset:

In [2]:
%%bash

rm -f dataset.hdf5
exetera import -w -s exeteraschema.json -i "participants:participants.csv, tests:tests.csv" -o dataset.hdf5
ls -lh

{'participants': 'participants.csv', 'tests': 'tests.csv'}
2021-06-15 22:49:25.288855+00:00
exeteraschema.json
{'participants': 'participants.csv', 'tests': 'tests.csv'}
loading took 2.765655517578125e-05 seconds
loading took 2.0503997802734375e-05 seconds
read_file_using_fast_csv_reader: 1 chunks, 10 accumulated_written_rows parsed in 0.9659640789031982s
completed in 0.9844956398010254 seconds
Total time 0.9845738410949707s
participants <KeysViewHDF5 ['participants']>
read_file_using_fast_csv_reader: 1 chunks, 10 accumulated_written_rows parsed in 0.058423757553100586s
completed in 0.0746314525604248 seconds
Total time 0.07473468780517578s
tests <KeysViewHDF5 ['participants', 'tests']>
<KeysViewHDF5 ['participants', 'tests']>
total 253M
-rw-r--r-- 1 localek10 bioeng 253M Jun 15 23:49 dataset.hdf5
-rw-r--r-- 1 localek10 bioeng  37K Jun 15 23:45 exetera_import.ipynb
-rw-r--r-- 1 localek10 bioeng 1.7K Jun 15 23:41 exeteraschema.json
-rw-r--r-- 1 localek10 bioeng  400 Jun 15 23:41 partici

Let's import ExeTera and read some of the data back:

In [4]:
with exetera.core.session.Session() as s:
    dat = s.open_dataset("dataset.hdf5", "r", "dataset")  # load the dataset
    print(list(dat))  # list the frames

    frame = dat["participants"]  # pull out a frame

    print(type(frame), len(frame))

    field = frame["FirstName"]  # pull out a field of the frame

    print(type(field), len(field))

    print(field.data)  # The "data" member is a proxy for the actual

    for i in range(len(field)):  # we can iterate over the data of the field
        print(i, field.data[i])

    age = frame["Age"]  # pull out another field
    age_filter = age >= 40  # create an array of boolean values to use as a selector

    print("Age filter data:", list(age_filter.data))

    print("Selected ages:", age.apply_filter(age_filter.data[:]).data[:])  # filter the field by the selector

    # filter all fields by the selector, saving as a dictionary
    filtered = {f: frame[f].apply_filter(age_filter.data[:]).data[:] for f in ["FirstName", "LastName", "Age", "id"]}
    # filtered=frame.apply_filter(age_filter.data[:]).items()  # since the dataset is read-only this will fail

    df = pd.DataFrame(filtered)  # convert to pandas for easy viewing

df

['participants', 'tests']
<class 'exetera.core.dataframe.HDF5DataFrame'> 8
<class 'exetera.core.fields.FixedStringField'> 10
0 b'hIggbeTBb7N'
1 b'tVRU4EmlSM'
2 b'g6GDAYH4'
3 b'unyvsyVj1V'
4 b'KAEUAhGu84qy'
5 b'SMJpRqfXoW'
6 b'30yuuy'
7 b'Jj3auoLwciJFI'
8 b'fjt1RDbKvKk0'
9 b'B3Jgae'
Age filter data: [True, True, True, True, False, True, False, True, True, True]
Selected ages: [87 88 77 60 89 68 64 78]


,FirstName,LastName,Age,id
0,b'hIggbeTBb7N',b'mkDtXPVpYSc2',87,b'0'
1,b'tVRU4EmlSM',b'2QITRSa',88,b'1'
2,b'g6GDAYH4',b'b40CWxb',77,b'2'
3,b'unyvsyVj1V',b'NDwsU5lFmA5E1',60,b'3'
4,b'SMJpRqfXoW',b'YUuXHbAeU2pR',89,b'5'
5,b'Jj3auoLwciJFI',b'Sk0Bxo8Ipj37',68,b'7'
6,b'fjt1RDbKvKk0',b'q8Kgf1G',64,b'8'
7,b'B3Jgae',b'rIQC5Grx',78,b'9'


Now we'll do the pointless thing of merging a dataset with itself, making a copy to preserve the original data:

In [5]:
def simple_merge(session, frame_left, frame_right, left_on, right_on, *other_fields, field_suffix="_R"):
    """
    Defines a simple merge between the left and right frames, using `left_on` and `right_on` as the keys to merge.
    If `other_fields` has names of fields these will be merged, if omitted all fields will be merged.
    """
    if len(other_fields) == 0:
        other_fields = list(frame_left)

    result = session.merge_left(
        left_on=frame_left[left_on],
        right_on=frame_right[right_on],
        right_fields=tuple(frame_right[f] for f in other_fields),
        right_writers=tuple(frame_right[f].create_like(frame_left, f + field_suffix) for f in other_fields),
    )

    return result


with exetera.core.session.Session() as s:
    src = s.open_dataset("dataset.hdf5", "r", "dataset")
    dest = s.open_dataset("datasetx2.hdf5", "w", "datasetx2")  # open the destination for writing

    sframe = src["participants"]  # pull out the frame to merge
    dest["participants"] = sframe  # create the same frame with data in the destination object
    dframe = dest["participants"]  # get that frame

    result = simple_merge(s, dframe, dframe, "id", "id", "FirstName", "Age", field_suffix="1")  # apply the merge

    df = pd.DataFrame({f: dframe[f].data[:] for f in dframe})  # convert to pandas

df

,Age,Age_valid,FirstName,LastName,id,is_employed,j_valid_from,j_valid_to,FirstName1,Age1
0,87,True,b'hIggbeTBb7N',b'mkDtXPVpYSc2',b'0',2,1.623797e+09,3.250368e+10,b'hIggbeTBb7N',87
1,88,True,b'tVRU4EmlSM',b'2QITRSa',b'1',1,1.623797e+09,3.250368e+10,b'tVRU4EmlSM',88
2,77,True,b'g6GDAYH4',b'b40CWxb',b'2',2,1.623797e+09,3.250368e+10,b'g6GDAYH4',77
3,60,True,b'unyvsyVj1V',b'NDwsU5lFmA5E1',b'3',1,1.623797e+09,3.250368e+10,b'unyvsyVj1V',60
4,24,True,b'KAEUAhGu84qy',b'IIBl3gqC1SaF',b'4',2,1.623797e+09,3.250368e+10,b'KAEUAhGu84qy',24
5,89,True,b'SMJpRqfXoW',b'YUuXHbAeU2pR',b'5',1,1.623797e+09,3.250368e+10,b'SMJpRqfXoW',89
6,33,True,b'30yuuy',b'stcGmU7IG',b'6',1,1.623797e+09,3.250368e+10,b'30yuuy',33
7,68,True,b'Jj3auoLwciJFI',b'Sk0Bxo8Ipj37',b'7',2,1.623797e+09,3.250368e+10,b'Jj3auoLwciJFI',68
8,64,True,b'fjt1RDbKvKk0',b'q8Kgf1G',b'8',1,1.623797e+09,3.250368e+10,b'fjt1RDbKvKk0',64
9,78,True,b'B3Jgae',b'rIQC5Grx',b'9',2,1.623797e+09,3.250368e+10,b'B3Jgae',78


Here we will create a new dataset in a new directory then merge that with our existing one:

In [7]:
%%bash

mkdir -p other_dataset
generate_dataset randomschema.yml other_dataset
cd other_dataset

rm -f dataset.hdf5
exetera import -w -s ../exeteraschema.json -i "participants:participants.csv, tests:tests.csv" -o dataset.hdf5

Schema: '<unopened file 'randomschema.yml' r>'
Output: '/home/localek10/workspace/RandomDataset/examples/other_dataset'
{'participants': 'participants.csv', 'tests': 'tests.csv'}
2021-06-15 23:26:48.209571+00:00
../exeteraschema.json
{'participants': 'participants.csv', 'tests': 'tests.csv'}
loading took 2.6941299438476562e-05 seconds
loading took 2.0265579223632812e-05 seconds
read_file_using_fast_csv_reader: 1 chunks, 10 accumulated_written_rows parsed in 0.9837634563446045s
completed in 1.0020349025726318 seconds
Total time 1.0021119117736816s
participants <KeysViewHDF5 ['participants']>
read_file_using_fast_csv_reader: 1 chunks, 10 accumulated_written_rows parsed in 0.06067204475402832s
completed in 0.07497644424438477 seconds
Total time 0.07507753372192383s
tests <KeysViewHDF5 ['participants', 'tests']>
<KeysViewHDF5 ['participants', 'tests']>


In [8]:
with exetera.core.session.Session() as s:
    src1 = s.open_dataset("dataset.hdf5", "r", "dataset")
    src2 = s.open_dataset("other_dataset/dataset.hdf5", "r", "otherdataset")
    dest = s.open_dataset("datasetx2.hdf5", "w", "datasetx2") 

    sframe = src1["participants"]
    dest["participants"] = sframe
    dframe = dest["participants"] 

    otherframe=src2["participants"]
    
    result = simple_merge(s, dframe, otherframe, "id", "id", "FirstName", "Age", field_suffix="1")

    df = pd.DataFrame({f: dframe[f].data[:] for f in dframe})

df

,Age,Age_valid,FirstName,LastName,id,is_employed,j_valid_from,j_valid_to,FirstName1,Age1
0,87,True,b'hIggbeTBb7N',b'mkDtXPVpYSc2',b'0',2,1.623797e+09,3.250368e+10,b'UThoOw',61
1,88,True,b'tVRU4EmlSM',b'2QITRSa',b'1',1,1.623797e+09,3.250368e+10,b'r5v2sG5gnLM',38
2,77,True,b'g6GDAYH4',b'b40CWxb',b'2',2,1.623797e+09,3.250368e+10,b'qB77yIMC6qJR',56
3,60,True,b'unyvsyVj1V',b'NDwsU5lFmA5E1',b'3',1,1.623797e+09,3.250368e+10,b'07Q7fTrkw',56
4,24,True,b'KAEUAhGu84qy',b'IIBl3gqC1SaF',b'4',2,1.623797e+09,3.250368e+10,b'mHmRgFg',64
5,89,True,b'SMJpRqfXoW',b'YUuXHbAeU2pR',b'5',1,1.623797e+09,3.250368e+10,b'qePoyPhk',82
6,33,True,b'30yuuy',b'stcGmU7IG',b'6',1,1.623797e+09,3.250368e+10,b'3ey5qfefMwg0A',18
7,68,True,b'Jj3auoLwciJFI',b'Sk0Bxo8Ipj37',b'7',2,1.623797e+09,3.250368e+10,b'YxiuvLaMCR',54
8,64,True,b'fjt1RDbKvKk0',b'q8Kgf1G',b'8',1,1.623797e+09,3.250368e+10,b'SXlWJi',52
9,78,True,b'B3Jgae',b'rIQC5Grx',b'9',2,1.623797e+09,3.250368e+10,b'NbFKAHDvOAT',57
